<a href="https://colab.research.google.com/github/yliao434/Musical-AI/blob/master/Musical_AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [0]:
# Dependencies required for Magenta.
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip uninstall -y tensorflow
!pip install -q magenta
!pip install -q ipywidgets
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

Uninstalling tensorflow-1.12.0:
  Successfully uninstalled tensorflow-1.12.0


#Import necessary libraries

In [0]:
from google.colab import files
import magenta.music as mm
import magenta
import tensorflow
from magenta.protobuf import music_pb2
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#Play a simple song

Some constants and helper functions

In [0]:
VELOCITY = 100
TIME_PER_TEMPO = 0.8 # Tempo of the song

def add_note(noteSequence, pitch, start_time, end_time):
  noteSequence.notes.add(pitch=pitch, start_time=start_time, end_time=end_time, velocity=VELOCITY) # Add new notes to the song
  
NOTE_NAME_TABLE = {'C':60, 'C#':61, 'D':62, 'D#':63, 'E':64, 'F':65, 'F#':66, 'G':67, 'G#':68, 'A':69, 'A#':70, 'B':71}  
def get_note_num(note_name, octave):
  return NOTE_NAME_TABLE[note_name] + octave * 12 # Convert alphabetical notation to number used in Magenta

Simple melody list

In [0]:
SIMPLE_MELODY = [('C', -1,1),('C', 0,1),('E', 0,1),('G', 0,1),('G', -2,1),('G', -1,1),('B', -1,1),('D', 0,1),
                ('A', -2,1),('A', -1,1),('C', 0,1),('E', 0,1),('E', -2,1),('E', -1,1),('G', -1,1),('B', -1,1),
                ('F', -2,1),('F', -1,1),('A', -1,1),('C', 0,1),('C', -2,1),('C', -1,1),('E', -1,1),('G', -1,1),
                ('F', -2,1),('F', -1,1),('A', -1,1),('C', 0,1),('G', -2,1),('G', -1,1),('B', -1,1),('D', 0,1), 
                ('C', -1,1),('C', 0,1),('E', 0,1),('G', 0,1),('G', -2,1),('G', -1,1),('B', -1,1),('D', 0,1),
                ('A', -2,1),('A', -1,1),('C', 0,1),('E', 0,1),('E', -2,1),('E', -1,1),('G', -1,1),('B', -1,1),
                ('F', -2,1),('F', -1,1),('A', -1,1),('C', 0,1),('C', -2,1),('C', -1,1),('E', -1,1),('G', -1,1),
                ('F', -2,1),('F', -1,1),('A', -1,1),('C', 0,1),('G', -2,1),('G', -1,1),('B', -1,1),('D', 0,1),('C', 0, 4)] # A list of s simple song

In [0]:
# Compose the song
simple_chord_progression = music_pb2.NoteSequence()

current_time = 0.0

for item in SIMPLE_MELODY:
  add_note(simple_chord_progression, get_note_num(item[0],item[1]), current_time, current_time + item[2] * TIME_PER_TEMPO)
  current_time += item[2] * TIME_PER_TEMPO

simple_chord_progression.total_time = current_time

simple_chord_progression.tempos.add(qpm=100);

mm.plot_sequence(simple_chord_progression) # Visualize the song

mm.play_sequence(simple_chord_progression,synth=mm.fluidsynth) # Play the song

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/pytho

#Generate music using pre-trained model

In [0]:
# Pre-trained model from Google.
pre_trained_model = TrainedModel(
      configs.CONFIG_MAP['cat-mel_2bar_big'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt') 

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'grad_clip': 1.0, 'z_size': 512, 'decay_rate': 0.9999, 'dec_rnn_size': [2048, 2048, 2048], 'free_bits': 0, 'use_cudnn': False, 'residual_encoder': False, 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'conditional': True, 'batch_size': 4, 'min_learning_rate': 1e-05, 'sampling_schedule': 'inverse_sigmoid', 'max_seq_len': 32, 'residual_decoder': False, 'beta_rate': 0.99999, 'enc_rnn_size': [2048], 'sampling_rate': 1000, 'max_beta': 0.5, 'dropout_keep_prob': 1.0, 'clip_mode': 'global_norm'}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Restoring parameters from /content/mel_2bar_big.ckpt


In [0]:
#@title Interative sliders to provide parameters to the generator
number_of_sequences_slider = 2 #@param {type:"slider", min:0, max:10, step:1}
temperature_slider = 2 #@param {type:"slider", min:-5, max:5, step:0.1}
length_slider = 100 #@param {type:"slider", min:0, max:200, step:1}

number_of_sequences = number_of_sequences_slider
temperature = temperature_slider
length = length_slider

Generate songs using pre-trained model

In [0]:
sequences = pre_trained_model.sample(n=number_of_sequences, length=length, temperature=temperature)

for item in sequences:
  mm.plot_sequence(item)
  mm.play_sequence(item, synth=mm.fluidsynth)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/pytho

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/pytho

#Use our model to generate

In [0]:
# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

bundle = mm.sequence_generator_bundle.read_bundle_file('model.mag')

# Initialize the model. The model is based on attention rnn.
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['attention_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

INFO:tensorflow:Restoring parameters from /tmp/tmp17O4UB/model.ckpt


Generate songs using our model

In [0]:
#@title Interative sliders to provide parameters to the generator
num_steps_slider = 512 #@param {type:"slider", min:200, max:525, step:1}
song_temperature_slider = 2 #@param {type:"slider", min:-5, max:5, step:0.1}

# How long the song is.
num_steps = num_steps_slider 

# How new song is different from the input.
song_temperature = song_temperature_slider 

In [0]:
# Ready to generate!

input_sequence = simple_chord_progression # My song

# Set the time. It will play the input sequences first then generated song.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)

# Set the tempo.
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

sequence = melody_rnn.generate(input_sequence, generator_options)

mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth=mm.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -1293.192993 


Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/pytho

#References:

##Some code in this project is from:
- Google, Hello Magenta.ipynb, https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb

##Songs for training the model:
- Timo Feng --- Zengirl
- Johann Pachelbel --- Canon
- Popcap --- Soundtrack of Plants vs. Zombies
- Dennis Kuo --- Crystal Rose
- Dennis Kuo --- Ever Eternity
- Dennis Kuo --- Sweet Saturn
- JJ Lin --- She Says
- The Piano Guys --- Love Story Meets Viva La Vida
- The Piano Guys --- Twinkle Lullaby

##Music data is from:
- Musescore: https://musescore.com/